## Dados demográficos e sobre a carreira

### Bibliotecas utilizadas

In [1]:
import pandas as pd
import altair as alt

pd.set_option('display.max_columns', 500)

### Dados

In [2]:
url_mapa = 'https://raw.githubusercontent.com/fititnt/gis-dataset-brasil/master/uf/geojson/uf.json' # dados geoespaciais dos estados do Brasil

In [3]:
df = pd.read_csv('../data/processed/state-of-data-2021.csv', low_memory=False)
df.tail()

,Idade,Genero,Estado,UF,Regiao,Nivel de Ensino,Area de Formacao,Cargo,Nivel,Faixa Salarial,Tempo de Experiencia,Forma de Trabalho
2640,40.0,Masculino,São Paulo,SP,Sudeste,Não tenho graduação formal,NaN,NaN,NaN,NaN,NaN,NaN
2641,27.0,Masculino,Santa Catarina,SC,Sul,Graduação/Bacharelado,Estatística/ Matemática / Matemática Computaci...,Cientista de Dados,Pleno,de R$ 6.001/mês a R$ 8.000/mês,de 4 a 5 anos,Modelo 100% presencial
2642,24.0,Masculino,Santa Catarina,SC,Sul,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,Engenheiro de Dados,Pleno,de R$ 6.001/mês a R$ 8.000/mês,de 2 a 3 anos,Modelo 100% remoto
2643,38.0,Masculino,Rio de Janeiro,RJ,Sudeste,Graduação/Bacharelado,Computação / Engenharia de Software / Sistemas...,NaN,NaN,NaN,NaN,NaN
2644,27.0,Feminino,Minas Gerais,MG,Sudeste,Pós-graduação,Outras,NaN,NaN,NaN,NaN,NaN


In [4]:
nomes_colunas = list(df.columns)
nomes_colunas

['Idade',
 'Genero',
 'Estado',
 'UF',
 'Regiao',
 'Nivel de Ensino',
 'Area de Formacao',
 'Cargo',
 'Nivel',
 'Faixa Salarial',
 'Tempo de Experiencia',
 'Forma de Trabalho']

In [5]:
df.isnull().sum()

Idade                    28
Genero                    0
Estado                   36
UF                       36
Regiao                    0
Nivel de Ensino           0
Area de Formacao         64
Cargo                   788
Nivel                   788
Faixa Salarial          280
Tempo de Experiencia    280
Forma de Trabalho       280
dtype: int64

In [6]:
df_atual = df.dropna() # dados sem valores nulos

### Perguntas sobre os dados

* Pergunta 1: Qual a idade média dos profissionais de ciência de dados, que ganham entre R$ 2.001,00 e R$ 8.000,00, do estado que possui a maior quantidade de profissionais de dados?
* Pergunta 2: Quanto tempo de experiência tem os profissionais do sexo feminino que ganham entre R$ 12.000,00 e R$ 20.000,00 e trabalham na região centro-oeste do Brasil?
* Pergunta 3: Qual o cargo dos profissionais de dados júnior que estão atualmente trabalhando no modelo 100% presencial, formados em computação com qualquer nível de ensino?

### Pergunta 1

In [7]:
df_agrado_mapa = df.groupby(["UF"]).count() # agrupar pela quantidade
df_agrado_mapa.reset_index(level=0, inplace=True) # criar uma coluna de UF, tirando UF do indíce

In [8]:
faixa_salarial_ordenada = ['Menos de R$ 1.000/mês',
                           'de R$ 1.001/mês a R$ 2.000/mês',
                           'de R$ 2.001/mês a R$ 3000/mês',
                           'de R$ 3.001/mês a R$ 4.000/mês',
                           'de R$ 4.001/mês a R$ 6.000/mês',
                           'de R$ 6.001/mês a R$ 8.000/mês',
                           'de R$ 8.001/mês a R$ 12.000/mês',
                           'de R$ 12.001/mês a R$ 16.000/mês',
                           'de R$ 16.001/mês a R$ 20.000/mês',
                           'de R$ 20.001/mês a R$ 25.000/mês',
                           'de R$ 25.001/mês a R$ 30.000/mês',
                           'de R$ 30.001/mês a R$ 40.000/mês',
                           'Acima de R$ 40.001/mês']

In [9]:
selecao_uf = alt.selection_multi(fields=['UF'], empty='all')
selecao_invervalo = alt.selection_interval(empty='all')

In [10]:
base = alt.Chart(alt.Data(url=url_mapa, format=alt.DataFormat(property='features',type='json'))).mark_geoshape(
        stroke="#555", strokeWidth=0.3, color="lightgray",
    ).transform_calculate(
        uf="datum.properties.UF_05"
    ).transform_lookup(
        lookup = "uf",
        from_ = alt.LookupData(data=df_agrado_mapa, key='UF', fields=list(df_agrado_mapa.columns))
    ).encode(
        tooltip = [alt.Tooltip('uf:N', title='Estado'),
                   alt.Tooltip('Genero:Q', title='Quantidade')],
                   opacity=alt.condition(selecao_uf, alt.value(1), alt.value(0.2))
    ).properties(
        width=400,
        height=400,
        title='Estados do Brasil',
    )

mapa = alt.layer(
    base, base.encode(color=alt.Color('Genero:Q', scale=alt.Scale(scheme='purples', reverse=False), title='Quantidade'))
    ).add_selection(selecao_uf)

In [11]:
cargo_faixa = alt.Chart(df_atual).mark_rect(tooltip=True).encode(
    
    y=alt.Y('Faixa Salarial:N', sort=faixa_salarial_ordenada, title='Faixa salarial'),
    x=alt.X('Cargo:N', title='Cargo'),
    color=alt.Color('count()', title='Quantidade', scale=alt.Scale(scheme='tealblues')),

    opacity=alt.condition(selecao_invervalo, alt.value(.9),alt.value(.1)),
    tooltip = [alt.Tooltip('Faixa Salarial:N', title='Faixa salarial'),
               alt.Tooltip('Cargo:N', title='Cargo'),
               alt.Tooltip('count()', title='Quantidade')]

).properties(
    title='Cargo por faixa salarial',
).transform_filter(
    selecao_uf
).add_selection(
    selecao_invervalo
)

In [12]:
linha = alt.Chart(df_atual).mark_rule(color='purple', strokeDash=[10, 10], tooltip=True).encode(
    
    y=alt.Y('mean(Idade):Q'),

    tooltip = [alt.Tooltip('mean(Idade):Q', title='Média')]

).transform_filter(selecao_uf).transform_filter(selecao_invervalo)

faixa_idade = alt.Chart(df_atual).mark_point(color='SeaGreen').encode(
    
    x=alt.X('Faixa Salarial:N', sort=faixa_salarial_ordenada, title='Faixa salarial'),
    y=alt.Y('mean(Idade):Q', scale=alt.Scale(zero=False), title='Idade média'),
    
    tooltip = [alt.Tooltip('Faixa Salarial:N', title='Faixa salarial'),
               alt.Tooltip('mean(Idade):Q', title='Idade média')]

).properties(
    width=400,
    height=200,
    title='Faixa salarial por idade média'
).transform_filter(selecao_uf).transform_filter(selecao_invervalo)

faixa_idade = alt.layer(faixa_idade, linha) # faixa_idade = faixa_idade+linha

In [13]:
# concatena horizontalmente o mapa e o mapa de calor
mapa_cargo_faixa = alt.hconcat(mapa, cargo_faixa.interactive()).resolve_scale(
    color='independent',
    size='independent'
)

# concatena verticalmente o gráfico de pontos aos dois anteriores
vis_pergunta_1 = alt.vconcat(mapa_cargo_faixa, faixa_idade.interactive()).resolve_scale(
    color='independent', 
    size='independent'
).configure_axisX(
    labelAngle=-45
)

vis_pergunta_1

alt.VConcatChart(...)

### Pergunta 2

In [14]:
lista_genero = list(df_atual['Genero'].unique())

print(lista_genero)

['Masculino', 'Feminino', 'Outro']


In [15]:
tempo_experiencia_ordenado = ['Não tenho experiência na área de dados',
                              'Menos de 1 ano',
                              'de 1 a 2 anos',
                              'de 2 a 3 anos',
                              'de 4 a 5 anos',
                              'de 6 a 10 anos',
                              'Mais de 10 anos']

In [16]:
input_dropdown = alt.binding_select(options=[None]+lista_genero, name='Gênero: ', labels=['Todos']+lista_genero)
botao_genero = alt.selection_single(fields=['Genero'], bind=input_dropdown, empty='all') # antigo botão

selecao_faixa_salarial = alt.selection_interval(empty='all')

In [17]:
faixa_salarial = alt.Chart(df_atual).mark_bar(color='#15AB98').encode(
    
    x=alt.X('count():Q', scale=alt.Scale(zero=False), title='Quantidade'),
    y=alt.Y('Faixa Salarial:N', sort=faixa_salarial_ordenada, title='Faixa Salarial'),

    opacity=alt.condition(selecao_faixa_salarial, alt.value(.9),alt.value(.1)),

    tooltip = [alt.Tooltip('Faixa Salarial:N', title='Faixa Salarial'),
               alt.Tooltip('count():Q', title='Quantidade')]

).properties(
    width=300,
    height=200,
    title='Resultados da faixa salarial',
).add_selection(
    botao_genero, selecao_faixa_salarial
).transform_filter(
    botao_genero
)

In [18]:
regiao_tempo_experiencia = alt.Chart(df_atual).mark_rect(tooltip=True).encode(
    
    y=alt.Y('Regiao:N', title='Região'),
    x=alt.X('Tempo de Experiencia:N', sort=tempo_experiencia_ordenado, title='Tempo de experiência', axis=alt.Axis(labelAngle=-45)),
    color=alt.Color('count()', title='Quantidade', scale=alt.Scale(scheme='purplered')),

    tooltip = [alt.Tooltip('Regiao:N', title='Região'),
               alt.Tooltip('Tempo de Experiencia:N', title='Tempo de experiência'),
               alt.Tooltip('count()', title='Quantidade')]

).properties(
    width=300,
    height=200,
    title='Tempo de experiência por região',
).transform_filter(botao_genero).transform_filter(selecao_faixa_salarial)

In [19]:
vis_pergunta_2 = alt.hconcat(faixa_salarial, regiao_tempo_experiencia.interactive()).resolve_scale(
    color='independent',
    size='independent'
)

vis_pergunta_2

alt.HConcatChart(...)

### Pergunta 3

In [20]:
nivel_ensino_ordenado = ['Estudante de Graduação',
                         'Graduação/Bacharelado',
                         'Pós-graduação',
                         'Mestrado',
                         'Doutorado ou Phd']

In [21]:
selecao_forma_nivel = alt.selection_multi(fields=['Forma de Trabalho'], empty='all', encodings=['x', 'color'])
selecao_area_nivel = alt.selection_interval(empty='all')

In [22]:
forma_trabalho = alt.Chart(df_atual).mark_bar(tooltip=True, color='#6A5ACD').encode(
    
    x=alt.X('Forma de Trabalho:N', title='Forma de trabalho', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('count():Q', title='Quantidade'),
    color=alt.Color('Nivel:N', title='Nível', scale=alt.Scale(range=['#fff7bc', '#fec44f', '#d95f0e'])),
    opacity=alt.condition(selecao_forma_nivel, alt.value(1), alt.value(0.2)),

).properties(
        width=300,
        height=300,
        title='Resultados da forma de trabalho'
).add_selection(selecao_forma_nivel)

In [23]:
area_nivel = alt.Chart(df_atual).mark_circle(tooltip=True).encode(
    
    y=alt.Y('Area de Formacao:N', title='Área de formação'),
    x=alt.X('Nivel de Ensino:N', sort=nivel_ensino_ordenado, title='Nível de ensino', axis=alt.Axis(labelAngle=-45)),
    color = alt.Color('count()', title='Quantidade', scale=alt.Scale(scheme='bluegreen')),
    size = alt.Size('count()', title='Quantidade'),
    opacity=alt.condition(selecao_area_nivel, alt.value(1), alt.value(0.2)),

).properties(
        width=300,
        height=300,
        title='Nível de ensino por área de formação'
).add_selection(selecao_area_nivel).transform_filter(selecao_forma_nivel)

In [24]:
cargo = alt.Chart(df_atual).mark_bar(tooltip=True, color='#483D8B').encode(
    
    y=alt.Y('Cargo:N', title='Cargo'),
    x=alt.X('count():Q', title='Quantidade')

).properties(
        width=300,
        height=200,
        title='Resultados do cargo'
).transform_filter(selecao_area_nivel).transform_filter(selecao_forma_nivel)

In [25]:
forma_trabalho_area_nivel = alt.hconcat(forma_trabalho, area_nivel.interactive()).resolve_scale(
    color='independent',
    size='independent'
)


vis_pergunta_3 = alt.vconcat(forma_trabalho_area_nivel, cargo.interactive()).resolve_scale(
    color='independent', 
    size='independent'
)

vis_pergunta_3

alt.VConcatChart(...)